In [ ]:
import sys

sys.path.append("..")
from common_utils import set_data_home

set_data_home("~/datasets")
from common_utils import DATA_HOME, join
from lstm.sales_data import Sales_Dataset
import torch
import random

device = "cuda" if torch.cuda.is_available() else "cpu"
SALE_HOME = join(DATA_HOME, "sales_data")
MODEL_HOME = join(DATA_HOME, "sale_model")

torch.manual_seed(42)
random.seed(42)

In [ ]:
sd = Sales_Dataset(SALE_HOME, seq_len=500, is_train=False)

In [ ]:
# from torch.nn import Transformer, Linear


# class MyTransformer(torch.nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.trans = Transformer(d_model=I, nhead=HEAD, batch_first=True).cuda()
#         self.linear = Linear(I, H)

#     def forward(self, src, tgt):
#         return self.linear(self.trans(src, tgt))
print(sd[0][0].shape, sd[1][0].shape, sd[2][0].shape)
I, H, B = 71, 34, 6
HEAD = 6
num_fams = 33

### Perform Inference

In [ ]:
import pandas as pd
from datetime import timedelta
from torch.nn import LSTM

INFERED_DAYS = 16

model = LSTM(I, H, batch_first=True).cuda()
model.load_state_dict(torch.load("sales_model_1000.pth"))
sales = pd.read_csv(join(SALE_HOME, "test.csv"), index_col=0)
sales["sales"] = 0.0
base_sales = sd.base_sales.set_index(["store_nbr", "date"])

for X, PROMO, O, store_id in sd:
    X = X.cuda()
    prev_sales = base_sales.loc[(store_id, sd.train_max_date)].sales
    prev_sales = torch.tensor(prev_sales.to_numpy(), dtype=torch.float32).cuda()
    next_x = torch.empty((1, I)).cuda()

    # infer and update input for each store
    for i in range(INFERED_DAYS):
        yhat, (_, _) = model(X)
        next_x[0, 0:66:2] = yhat[-1, :33]  # sales
        next_x[0, 1:66:2] = PROMO[i]  # promotion
        next_x[0, 66] = O[i]  # oil
        next_x[0, 67] = yhat[-1, 33]  # transaction
        next_x[0, 68:71] = X[0, 68:71]  # geolocations
        # next_x[0, 71] = 0.0

        X = torch.concat((X[1:], next_x))

        # compute actual sales
        yhat_rets = yhat[-1][:-1]  # get all columns but last (transaction)
        curr_sales = sd.ret_2_sale(yhat_rets, prev_sales)
        ts = (sd.train_max_date + timedelta(days=i + 1)).strftime("%Y-%m-%d")

        # write to each family in the answer dataframe
        for j, f in enumerate(sd.families):
            sales.loc[
                (sales.date == ts)
                & (sales.store_nbr == store_id)
                & (sales.family == f),
                "sales",
            ] = (
                curr_sales[j].cpu().item()
            )

### output the answer

In [ ]:
sales

In [ ]:
sales.drop(columns=["store_nbr", "date", "family", "onpromotion"]).sort_values(
    "id"
).sort_index().to_csv("answer.csv", index=True)